In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [6]:
# Set the URL of the page
url = "https://realtylink.org/en/properties~for-rent"

# Set up the Chrome WebDriver
driver = webdriver.Chrome()
# Navigate to the initial page
driver.get(url)
prev_page = 0    
links = []
# Iterate through pages
retries = 0
while retries <= 3:
    for i in range(0, 6):
        start_time = time.monotonic()
        while True:
            pager_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'pager-current'))
            )
            current_page_text = pager_element.text
            current_page = int(current_page_text.split('/')[0].strip())
        
            if current_page - prev_page == 1:
                # Extract data from the current page if the page have changed
                page = driver.page_source
                # Parse the HTML content
                soup = BeautifulSoup(page, 'html.parser')
                # Find all the <a> elements with class 'property-thumbnail-summary-link'
                link_elements = soup.find_all('a', class_='property-thumbnail-summary-link')
                # Extract the 'href' attribute from each <a> element
                links.extend([link['href'] for link in link_elements])
                prev_page = current_page
                break

            end_time = time.monotonic()        
            if end_time - start_time > 10:
                raise TimeoutError("Timeout!")
            

        # Find and click the "Next" button
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'li.next a'))
            )
            next_button.click()
        except Exception as e:
            # Break the loop if the "Next" button is not found
            print(f"Error: {e}")
            retries += 1
            break
    break

# Close the WebDriver
# driver.quit()

In [8]:
"data"[:-1]

'dat'

In [9]:
data = []
url = "https://realtylink.org"
for link in links[22:]:
    page_link = url + link
    driver.get(url = page_link)
    pager_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'pager-current'))
    )
    page_source = driver.page_source
    # Locate the <span> element by its attribute 'data-id'
    title_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//span[@data-id="PageTitle"]'))
    )
    # Get the text content of the <span> element
    title = title_element.text

    # Find the <h2> element based on its XPath
    address_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//h2[@itemprop="address" and contains(@class, "pt-1")]'))
    )
    # Get the text content of the <span> element
    address = address_element.text
    address_split = address.split(', ')
    for element in address_split[1:]:
        region = element + ", "
    region = region[:-1]

    try:
        description_element = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.XPATH, '//div[@itemprop="description"]'))
        )
        description = description_element.text
    except:
        description = None
    price_element = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'price.text-right'))
    )
    price = price_element.text

    try:
        bedrooms_element = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'cac'))
        )
        bedrooms = int(bedrooms_element.text.split(' ')[0])
    except:
        bedrooms = None
    try:
        bathrooms_element = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'sdb'))
        )
        bathrooms = int(bathrooms_element.text.split(' ')[0])
    except:
        bathrooms = None        

    area_element = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'carac-value'))
    )
    area = int(area_element.text.split(' ')[0].replace(',', ''))
    
    image_button = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'primary-photo-container'))
    )
    image_button.click()

    image_panel = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'carousel'))
    )
    image_urls = []
    stop = False
    while not stop:
        # print("1")
        # # try:
        img = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'fullImg'))
        )
        image_urls.append(img.get_attribute("src"))
        # Create an ActionChains object
        actions = ActionChains(driver)

        # Press the right arrow key
        actions.send_keys(Keys.ARROW_RIGHT)

        # Perform the action
        actions.perform()
        img_retries = 0
        while img_retries <= 5:
            try:
                desc_element = WebDriverWait(driver, 1).until(
                    EC.presence_of_element_located((By.XPATH, '//div[@class="description"]/strong'))
                )
                desc = desc_element.text.split("/")
                if desc[0] == desc[1]:
                    stop = True
                break
            except:
                img_retries += 1
        # next_button = WebDriverWait(driver, 1).until(
        #     EC.presence_of_element_located((By.CLASS_NAME, 'activateNextArrow'))
        # )
        # next_button.click()
        # except:
        #     break
        
    # # Find all <li> elements within the <ul> element
    # li_elements = image_panel.find_elements(By.TAG_NAME, "li")

    # # Extract the src attribute of each <img> element within <li> elements
    # image_urls.extend([li.find_element(By.TAG_NAME, "img").get_attribute("src") for li in li_elements])

    last_check_date = pd.Timestamp(datetime.now())

    page_data = {
        "page_link": page_link,
        "title": title,
        "address": address,
        "region": region,
        "description": description,
        "price": price,
        'bedrooms': bedrooms,
        "bathrooms": bathrooms,
        "area": area,
        "image_urls": image_urls,
        "last_check_date": last_check_date,
    }
    data.append(page_data)
    time.sleep(1)
    # break

KeyboardInterrupt: 

In [279]:
desc_element = WebDriverWait(driver, 1).until(
    EC.presence_of_element_located((By.XPATH, '//div[@class="description"]/strong'))
)
desc = desc_element.text.split("/")

In [11]:
pd.DataFrame(data)

,page_link,title,address,region,description,price,bedrooms,bathrooms,area,image_urls,last_check_date
0,https://realtylink.org/en/house~for-rent~white...,House for rent,"A 15622 Thirft Avenue, White Rock","White Rock,",UPDATED upper 3 bedroom recently updated with ...,"$2,800 /month",3,1,1450,"[None, https://mediaserver.realtylink.org/medi...",2024-01-14 16:52:03.703661
1,https://realtylink.org/en/apartment~for-rent~y...,Apartment for rent,"618 1088 Richards Street, Yaletown, Vancouver","Vancouver,",None,"$3,500 /month",2,1,657,"[None, https://mediaserver.realtylink.org/medi...",2024-01-14 16:52:07.580217
2,https://realtylink.org/en/apartment~for-rent~s...,Apartment for rent,"1108 3430 East Ken Avenue, South Marine, Vanco...","Vancouver,","Vancouver river district Paradigm building, Br...","$2,450 /month",1,1,526,"[None, https://mediaserver.realtylink.org/medi...",2024-01-14 16:52:11.179509
3,https://realtylink.org/en/apartment~for-rent~y...,Apartment for rent,"706 928 Beatty Street, Yaletown, Vancouver","Vancouver,",None,"$2,690 /month",0,1,443,"[None, https://mediaserver.realtylink.org/medi...",2024-01-14 16:52:16.185953
4,https://realtylink.org/en/apartment~for-rent~w...,Apartment for rent,"18 935 Jervis Street, West End VW, Vancouver","Vancouver,",None,"$2,850 /month",1,1,695,"[None, https://mediaserver.realtylink.org/medi...",2024-01-14 16:52:20.315616
5,https://realtylink.org/en/apartment~for-rent~w...,Apartment for rent,"1200 Alberni Street, West End VW, Vancouver","Vancouver,",Welcome to the Palisades in Downtown Vancouver...,"$3,250 /month",2,2,888,"[None, https://mediaserver.realtylink.org/medi...",2024-01-14 16:52:23.026561
6,https://realtylink.org/en/apartment~for-rent~d...,Apartment for rent,"516 68 Smithe Street, Downtown VW, Vancouver","Vancouver,",None,"$3,200 /month",1,1,505,"[None, https://mediaserver.realtylink.org/medi...",2024-01-14 16:52:26.947865
7,https://realtylink.org/en/house~for-rent~south...,House for rent,"Half-Basement 5968 Churchill Street, South Gra...","Vancouver,",Rare on market in prime Vancouver neighbourhoo...,"$3,300 /month",2,2,2000,"[None, https://mediaserver.realtylink.org/medi...",2024-01-14 16:52:30.204014
8,https://realtylink.org/en/apartment~for-rent~w...,Apartment for rent,"301 1168 Pendrell Street, West End VW, Vancouver","Vancouver,",None,"$3,900 /month",2,2,705,"[None, https://mediaserver.realtylink.org/medi...",2024-01-14 16:52:34.241382
9,https://realtylink.org/en/apartment~for-rent~w...,Apartment for rent,"15xx 8199 Capstan Way, West Cambie, Richmond","Richmond,","Viewstar, the largest waterfront community in ...","$3,500 /month",2,2,960,"[None, https://mediaserver.realtylink.org/medi...",2024-01-14 16:52:37.715164


In [260]:
next_button = WebDriverWait(driver, 1).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'activateNextArrow'))
)
next_button.click()

In [243]:
image_urls

['https://mediaserver.realtylink.org/media.ashx?id=ADDD252D2BF4466DDDDDDDDDD0&t=pi&w=100&h=75&sm=c',
 'https://mediaserver.realtylink.org/media.ashx?id=ADDD252D2BF42DDDDDDDDDDDDC&t=pi&w=100&h=75&sm=c',
 'https://mediaserver.realtylink.org/media.ashx?id=ADDD252D2BF42D1DDDDDDDDDDE&t=pi&w=100&h=75&sm=c',
 'https://mediaserver.realtylink.org/media.ashx?id=ADDD252D2BF42D4DDDDDDDDDDA&t=pi&w=100&h=75&sm=c',
 'https://mediaserver.realtylink.org/media.ashx?id=ADDD252D2BF42D2DDDDDDDDDD0&t=pi&w=100&h=75&sm=c',
 'https://mediaserver.realtylink.org/media.ashx?id=ADDD252D2BF42DCDDDDDDDDDDB&t=pi&w=100&h=75&sm=c',
 'https://mediaserver.realtylink.org/media.ashx?id=ADDD252D2BF42DEDDDDDDDDDDF&t=pi&w=100&h=75&sm=c',
 'https://mediaserver.realtylink.org/media.ashx?id=ADDD252D2BF42DADDDDDDDDDDD&t=pi&w=100&h=75&sm=c',
 'https://mediaserver.realtylink.org/media.ashx?id=ADDD252D2BF42D0DDDDDDDDDD1&t=pi&w=100&h=75&sm=c',
 'https://mediaserver.realtylink.org/media.ashx?id=ADDD252D2BF42DBDDDDDDDDDD4&t=pi&w=100&h=

In [194]:
date_element.get_attribute("value")

''

In [174]:
description

"Discover the perfect blend of convenience and comfort in this brand-new townhouse. With 4 bedrooms and 2.5 baths, it's an ideal space for modern living. Located in Surrey's Sullivan Station neighbourhood, you're just a short drive from schools and shops.The pristine condition of this residence welcomes you as its first Tenant. Indulge in a culinary haven with our townhouse featuring a high-ceiling kitchen adorned with stainless steel appliances. Pets allowed W/Restrictions. Please call Happy Boyal for showings and application at 778-859-3453. Credit/ Reference/ Employment check will be done after receiving signed application."

In [139]:
requests.get("https://realtylink.org/en/townhouse~for-rent~downtown-vw-vancouver/262861913?view=Summary&uc=4")

<Response [403]>

In [ ]:


# Make a GET request to the webpage
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all <a> elements with the specified attribute
    links_with_data_page = soup.find_all('a', {'data-page': True})

    all_pages = []
    # Extract and print the data-page attributes
    for link in links_with_data_page:
        if link.text.isdigit():
            all_pages.append(int(link.text))
    
    max_page = max(all_pages)

    parsed = []
    for i in range(1, max_page):
        print(i)
        print(len(parsed))
        if len(parsed) >= 100: #TEMP
            df = pd.DataFrame(parsed)
            job = CLIENT.load_table_from_dataframe(df, tableRef, job_config=job_config)  # Make an API request.
            job.result()  # Wait for the job to complete.

            table = CLIENT.get_table(tableRef)  # Make an API request.
            echo = "Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), tableRef)
            print(echo)
            parsed = []
            # break
        
        url = f"https://data.gov.ua/global_search?q=%D0%94%D0%B0%D0%BD%D1%96+%D0%BF%D1%80%D0%BE+%D0%B4%D0%B5%D0%BF%D1%83%D1%82%D0%B0%D1%82%D1%96%D0%B2&type=all&page={i}"
        # send a GET request to the URL and get the HTML content
        retries = 0
        while True:
            if retries >= 10:
                raise Exception("Bad response 2")
            response = requests.get(url)
            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                # Parse the HTML content of the page
                soup = BeautifulSoup(response.text, 'html.parser')

                # Find all <div> elements with the specified class
                div_elements = soup.find_all('div', class_='search-container__item')

                # Extract and print information from each <div> element
                # names = []
                # descriptions = []
                # urls = []
                for div_element in div_elements:
                    # Extract text from the <h4> element within the current <div>
                    name = div_element.find('h4', class_='search-container__item-name').text.strip()
                    if "дані про депутатів" in name.lower():
                        # Extract text from the <p> element within the current <div>
                        description = div_element.find('p', class_='search-container__item-description').text.strip()
                        url = div_element.find('a', href=True)['href']
                        page = i
                        timestamp = pd.Timestamp(datetime.now())
                        parsed.append({"name": name, "description": description, "url": url, "page": page, "timestamp": timestamp})
                break
                
                # break #TEMP
            else:
                retries += 1
                # raise Exception("Bad response 2")
else:
    raise Exception("Bad response 1")